# https://retailcontentservice.com/catalog/

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import warnings
warnings.simplefilter("ignore")
import time
from itertools import groupby
import os
import random
import json
import datetime

headers={
    
    }

def get_links(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'lxml')
    links=soup.find_all('a',class_='btn-block')
    main_links=[]
    
    for i in links:
        main_links.append(str('https://retailcontentservice.com')+str(i.get('href')))
#     print(main_links[1:16])
    result=[]
    
    for j in main_links[1:16]:       
        response=requests.get(j)
        soup=BeautifulSoup(response.text,'lxml')
        links=soup.find('ul',class_='link-list link-list_md').find_all('li')
        category=soup.find('h1').text.strip()
        sub_category=[]
        second_links=[]
        
        
        
        for i in links:
            second_links.append(str('https://retailcontentservice.com')+str(i.find('a').get('href')))
            sub_category.append(i.find('a').text.strip())
            
        for i in range(len(second_links)):
            result.append({
                'category':category,
                'sub_category':sub_category[i],
                'links':second_links[i]
                
                
                
            })
        
            
    result2=[]
    for i in result:
        response=requests.get(i.get('links'))
        soup=BeautifulSoup(response.text,'lxml')
        links2=soup.find('ul',class_='link-list link-list_md').find_all('li')
        second_links2=[]
        sub_category2=[]
        for j in links2:
            second_links2.append(str('https://retailcontentservice.com')+str(j.find('a').get('href')))
            sub_category2.append(j.find('a').text.strip())
        
        for k in range(len(second_links2)):
            
            result2.append({                
            'category':i.get('category'),
            'sub_category':i.get('sub_category'),
            'sub_sub_category':sub_category2[k],     
            'links':second_links2[k]          
                       
        })
    
    with open(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\barcode\retailcontentservice.com\main_url.json','w',encoding='utf-8') as file:
                json.dump(result2,file,indent=4,ensure_ascii=False)        
            
def collect_data(file_path):
    data = json.load(open(file_path,'r', encoding='utf-8'))
    result=[]
    
    for i in data:
        links=[]
        
        for j in range(1,20):
            
            response=requests.get(str(i.get('links'))+'page'+str(j)+'/',headers=headers)
            soup=BeautifulSoup(response.text,'lxml')
            link=soup.find_all('div',class_='catalog__grid-view__item products-slider__item col-sm-6 wrap wrap-sm-2 col-lg-4 wrap-lg-3')
            
            if link!=None:
                for k in link:
                    links.append(str('https://retailcontentservice.com/')+str(k.find('a').get('href')))
            else:                
                break
        for k in range(len(links)):
            result.append({                
            'category':i.get('category'),
            'sub_category':i.get('sub_category'),
            'sub_sub_category':i.get('sub_sub_category'),     
            'links':links[k]          
                       
        })
    with open(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\barcode\retailcontentservice.com\url.json','w',encoding='utf-8') as file:
                json.dump(result,file,indent=4,ensure_ascii=False)  
                
def collect_all_dates(file_path):
    result=[]
    data = json.load(open(file_path,'r', encoding='utf-8'))
    for i in data[:20]:
        response=requests.get(i.get('links'),headers=headers)
        soup=BeautifulSoup(response.text,'lxml')        
        titles=[]
        brands=[]
        producers=[]
        barcodes=[]
        try:
            titles.append(soup.find('h1').text.strip())
        except:
            titles.append(None)

        try:
            brand=soup.find_all('ul',class_='list-unstyled list-lg-pad')
            for j in brand:
                brands.append(j.find('li').text)
        except:
            brands.append(None)
            
        try:
            barcode=soup.find_all(class_='col-xs-6 col-sm-6 col-md-6 col-lg-12 bcTarget')
            for k in barcode:
                barcodes.append(k.get('gtin'))
        except:
            barcodes.append(None)
            
        try:
            producers.append(soup.find('div',class_='wrap-name-main-merchant').text.strip())
        except:
            producers.append(None)
            
        
        result.append({ 
            'name':titles[0],
            'barcode':barcodes,
            'brand':brands[0],
            'producer':producers[0],
            'category':i.get('category'),
            'sub_category':i.get('sub_category'),
            'sub_sub_category':i.get('sub_sub_category'),     
            'links':i.get('links')         
                       
        })
    with open(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\barcode\retailcontentservice.com\all_data.json','w',encoding='utf-8') as file:
                json.dump(result,file,indent=4,ensure_ascii=False)  
            
            

def main():
    #get_links('https://retailcontentservice.com/catalog/')
    #collect_data(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\barcode\retailcontentservice.com\main_url.json')
    collect_all_dates(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\barcode\retailcontentservice.com\url.json')
    
if __name__=="__main__":
     main()